# **Pencarian dan Penambangan Web - Tugas 2: Membuat VSM Dengan Isi TF-IDF Berdasarkan Hasil Crawling Berita**

Dibuat Oleh:

Nama: Muhammad Fahmi

Nim: 190411100061

Kelas: PPW A

### Persiapan Library

In [1]:
!pip install -q Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.9 MB/s eta 0:00:00


Bagian ini bertujuan untuk menginstal pustaka Sastrawi, sebuah pustaka untuk melakukan stemming pada teks berbahasa Indonesia. Stemming adalah proses mengubah kata-kata ke bentuk dasarnya. -q digunakan untuk mengurangi keluaran agar lebih ringkas.

In [2]:
import numpy as np
import re
import pandas as pd

from tqdm import tqdm_gui

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


*   `numpy (np)`: Digunakan untuk operasi numerik dan manipulasi array.
*   `re`: Pustaka untuk manipulasi dan pencocokan pola teks menggunakan regular expression.
*   `pandas (pd)`: Digunakan untuk memanipulasi data, terutama data berbentuk tabel (data frame).
*   `tqdm_gui`: Digunakan untuk menampilkan progress bar yang interaktif ketika memproses data dalam jumlah besar.
*   `stopwords`: Kumpulan kata umum seperti "dan", "atau", "yang" yang sering dihapus dalam proses analisis teks.
*  ` punkt`: Modul untuk pemotongan teks menjadi kalimat atau kata.
*   `wordnet`: Kumpulan sinonim kata yang digunakan untuk memproses dan memahami makna kata.
*   `stopwords`: Mengambil daftar kata umum (stopwords) dari NLTK untuk digunakan dalam pengolahan teks.
*   `StemmerFactory`: Membuat stemmer untuk bahasa Indonesia dari Sastrawi.
*   `TfidfTransformer`: Digunakan untuk mengubah frekuensi kata (count) menjadi nilai TF-IDF (Term Frequency-Inverse Document Frequency), yang menyoroti kata yang lebih penting dalam kumpulan dokumen.
*   `CountVectorizer`: Mengubah teks menjadi matriks frekuensi kata.
*   `Pipeline`: Menggabungkan beberapa langkah pemrosesan data ke dalam satu alur yang bisa digunakan bersama.





### Mengambil Data Dari File CSV

In [4]:
main_df = pd.read_csv('https://raw.githubusercontent.com/Muhammadfahmi1806/PPW/main/tugas-2/hasil_crawling.csv') # Changed the URL to point to the raw content of the CSV file.
main_df

,Judul,Kategori,Tanggal,Isi
0,Angkringan Pak Aam Pringgondani Jogja: Pernah ...,Kuliner,10 September 2024,"Umumnya, angkringan dianggap sebagai jujugan a..."
1,Cerita Penjual Some Jawa Pertama di Jogja: Set...,Kuliner,10 September 2024,Sudah 50 tahun Roso (77) menjual some Jawa ata...
2,Merasakan Kuliner Aceh yang Autentik dan Merak...,Kuliner,7 September 2024,Warkop Aceh Bang Zainal yang terletak di pingg...
3,Kerja di Pasar Minggu Jakarta Selatan Bikin Ta...,Kuliner,4 September 2024,"Kerja di Pasar Minggu, Jakarta Selatan, sebena..."
4,Warung Makan Barokah Sih Surabaya: Rp8 Ribu Su...,Kuliner,23 Agustus 2024,Di antara semakin banyaknya warung makan yang ...
...,...,...,...,...
95,Buang Biaya Kuliah IPB Rp30 Juta Demi Pindah k...,Kampus,24 Juni 2024,"Sudah diterima kuliah di IPB, kampus cukup fav..."
96,Mahasiswa Asal Surabaya Nekat Kuliah HI UB Mal...,Kampus,22 Juni 2024,"Demi memenuhi keinginan orang tua, mahasiswa a..."
97,"Penyesalan Tolak IPB Demi Masuk ITS Surabaya, ...",Kampus,22 Juni 2024,"Awalnya bermimpi masuk IPB, begitu diterima, a..."
98,Mahasiswa PTN Surabaya Lulus Sarjana di Semest...,Kampus,21 Juni 2024,"Menjelang batas waktu drop out (DO), mahasiswa..."



Kode ini membaca data dari file CSV yang terletak di URL tertentu dan menyimpannya ke dalam sebuah DataFrame menggunakan pustaka pandas.

### Membuat Fungsi

In [6]:
# Case Folding
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

# Menghapus tanda baca, angka, dan simbol
def clean_punct(text):
    clean_spcl = re.compile('[/(){}\|@,;_]')
    clean_symbol = re.compile('[^0-9a-z]')
    clean_number = re.compile('[0-9]')
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    text = clean_number.sub('', text)
    return text

# Menghaps double atau lebih whitespace
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

# Menghapus stopwords
def clean_stopwords(text):
    stopword = set(stopwords.words('indonesian'))
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text

# Stemming with Sastrawi
def sastrawistemmer(text):
    factory = StemmerFactory()
    st = factory.create_stemmer()
    text = ' '.join(st.stem(word) for word in tqdm(text.split()) if word in text)
    return text

Kode ini berisi beberapa fungsi untuk membersihkan dan memproses teks dalam bahasa Indonesia, yang digunakan dalam Natural Language Processing (NLP):

* `clean_lower(lwr)`: Mengubah semua huruf dalam teks menjadi huruf kecil (lowercase) untuk konsistensi.

* `clean_punct(text)`: Menghapus tanda baca, angka, dan simbol khusus dari teks.

* `_normalize_whitespace(text)`: Menghapus spasi ganda atau lebih dan mengganti tab atau baris baru yang berulang menjadi satu, serta menghapus spasi yang tidak diperlukan di awal dan akhir teks.

* `clean_stopwords(text)`: Menghapus stopwords (kata umum seperti "yang", "dan", "atau") dari teks untuk menyederhanakan analisis. Daftar stopwords diambil dari pustaka NLTK untuk bahasa Indonesia.

* `sastrawistemmer(text)`: Melakukan stemming pada teks, yaitu mengubah kata-kata menjadi bentuk dasar menggunakan pustaka Sastrawi. Proses ini dilakukan kata per kata dengan bantuan progress bar dari tqdm untuk menunjukkan kemajuan.

### Mulai Praproses Teks

In [7]:
main_df['lwr'] = main_df['Isi'].apply(clean_lower)
casefolding=pd.DataFrame(main_df['lwr'])
casefolding

,lwr
0,"umumnya, angkringan dianggap sebagai jujugan a..."
1,sudah 50 tahun roso (77) menjual some jawa ata...
2,warkop aceh bang zainal yang terletak di pingg...
3,"kerja di pasar minggu, jakarta selatan, sebena..."
4,di antara semakin banyaknya warung makan yang ...
...,...
95,"sudah diterima kuliah di ipb, kampus cukup fav..."
96,"demi memenuhi keinginan orang tua, mahasiswa a..."
97,"awalnya bermimpi masuk ipb, begitu diterima, a..."
98,"menjelang batas waktu drop out (do), mahasiswa..."


Kode ini melakukan proses case folding (mengubah teks menjadi huruf kecil) pada kolom "Isi" dari DataFrame main_df:

* `main_df['lwr'] = main_df['Isi'].apply(clean_lower)`: Menggunakan fungsi clean_lower untuk mengubah semua teks di kolom "Isi" menjadi huruf kecil dan menyimpannya ke dalam kolom baru bernama "lwr".

* `casefolding = pd.DataFrame(main_df['lwr'])`: Membuat DataFrame baru bernama casefolding yang hanya berisi kolom "lwr" dari main_df, yaitu teks yang sudah diubah menjadi huruf kecil.

In [8]:
main_df['clean_punct'] = main_df['lwr'].apply(clean_punct)
main_df['clean_punct']

,clean_punct
0,umumnya angkringan dianggap sebagai jujugan an...
1,sudah tahun roso menjual some jawa atau some...
2,warkop aceh bang zainal yang terletak di pingg...
3,kerja di pasar minggu jakarta selatan sebenarn...
4,di antara semakin banyaknya warung makan yang ...
...,...
95,sudah diterima kuliah di ipb kampus cukup favo...
96,demi memenuhi keinginan orang tua mahasiswa as...
97,awalnya bermimpi masuk ipb begitu diterima ada...
98,menjelang batas waktu drop out do mahasiswa pt...



Kode ini membersihkan teks dari tanda baca, angka, dan simbol di kolom "lwr" dari DataFrame main_df:

* `main_df['clean_punct'] = main_df['lwr'].apply(clean_punct)`: Menggunakan fungsi clean_punct untuk menghapus tanda baca, angka, dan simbol dari teks di kolom "lwr" dan menyimpannya ke dalam kolom baru bernama "clean_punct".

* `main_df['clean_punct']`: Menampilkan isi kolom "clean_punct", yaitu teks yang sudah dibersihkan dari tanda baca, angka, dan simbol.

In [9]:
main_df['clean_double_ws'] = main_df['clean_punct'].apply(_normalize_whitespace)
main_df['clean_double_ws']

,clean_double_ws
0,umumnya angkringan dianggap sebagai jujugan an...
1,sudah tahun roso menjual some jawa atau some n...
2,warkop aceh bang zainal yang terletak di pingg...
3,kerja di pasar minggu jakarta selatan sebenarn...
4,di antara semakin banyaknya warung makan yang ...
...,...
95,sudah diterima kuliah di ipb kampus cukup favo...
96,demi memenuhi keinginan orang tua mahasiswa as...
97,awalnya bermimpi masuk ipb begitu diterima ada...
98,menjelang batas waktu drop out do mahasiswa pt...


Kode ini menghilangkan spasi ganda atau lebih dari teks di kolom "clean_punct" dalam DataFrame main_df:

* `main_df['clean_double_ws'] = main_df['clean_punct'].apply(_normalize_whitespace)`: Menggunakan fungsi _normalize_whitespace untuk menghapus spasi berlebih (spasi ganda atau lebih) dari teks di kolom "clean_punct" dan menyimpannya ke dalam kolom baru bernama "clean_double_ws".

* `main_df['clean_double_ws']`: Menampilkan isi kolom "clean_double_ws", yaitu teks yang sudah dibersihkan dari spasi yang berlebihan.

In [10]:
main_df['clean_sw'] = main_df['clean_double_ws'].apply(clean_stopwords)
main_df['clean_sw']

,clean_sw
0,angkringan dianggap jujugan anak muda jogja no...
1,roso menjual some jawa some ndeso sepeda tuany...
2,warkop aceh bang zainal terletak pinggiran ber...
3,kerja pasar minggu jakarta selatan menjanjikan...
4,banyaknya warung makan bertebaran unair kampus...
...,...
95,diterima kuliah ipb kampus favorit indonesia m...
96,memenuhi orang tua mahasiswa surabaya nekat ku...
97,bermimpi masuk ipb diterima situasi membuatnya...
98,menjelang batas drop out do mahasiswa ptn sura...


Kode ini menghapus stopwords (kata-kata umum yang tidak terlalu penting) dari teks di kolom "clean_double_ws" dalam DataFrame main_df:

* `main_df['clean_sw'] = main_df['clean_double_ws'].apply(clean_stopwords)`: Menggunakan fungsi clean_stopwords untuk menghapus stopwords dari teks di kolom "clean_double_ws" dan menyimpannya ke dalam kolom baru bernama "clean_sw".

* `main_df['clean_sw']`: Menampilkan isi kolom "clean_sw", yaitu teks yang sudah dihapus stopwords-nya.

In [12]:
!pip install tqdm
from tqdm import tqdm

def sastrawistemmer(text):
    factory = StemmerFactory()
    st = factory.create_stemmer()
    text = ' '.join(st.stem(word) for word in tqdm(text.split()) if word in text)
    return text

Kode ini melakukan stemming (mengubah kata-kata menjadi bentuk dasarnya) pada teks menggunakan pustaka Sastrawi, dengan tambahan progress bar untuk menunjukkan kemajuan proses

In [13]:
main_df['desc_clean_stem'] = main_df['clean_sw'].apply(sastrawistemmer)
main_df['desc_clean_stem']

100%|██████████| 569/569 [00:12<00:00, 45.70it/s]


,desc_clean_stem
0,angkring anggap jujugan anak muda jogja nongkr...
1,roso jual some jawa some ndeso sepeda tua alam...
2,warkop aceh bang zainal letak pinggir usaha ha...
3,kerja pasar minggu jakarta selatan janji gaji ...
4,banyak warung makan tebar unair kampus b surab...
...,...
95,terima kuliah ipb kampus favorit indonesia pil...
96,penuh orang tua mahasiswa surabaya nekat kulia...
97,mimpi masuk ipb terima situasi buat paksa pili...
98,jelang batas drop out do mahasiswa ptn surabay...


Kode ini mengubah teks yang telah dibersihkan dari stopwords menjadi bentuk dasar kata (stemming) dan menyimpannya untuk analisis lebih lanjut.

### Pembentukan Vector Space Model

In [14]:
count_vectorizer = CountVectorizer()
corpus = main_df['desc_clean_stem'].tolist()
x_count = count_vectorizer.fit_transform(corpus)
print("Shape:", x_count.shape)

Shape: (100, 4796)


Kode ini menghitung frekuensi kemunculan kata dalam teks dan mengubahnya menjadi matriks fitur

In [15]:
vocabulary = count_vectorizer.get_feature_names_out().tolist()
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                 ('tfid', TfidfTransformer())]).fit(corpus)

tf = pipe['count'].transform(corpus).toarray()
tfidf = []
for row in tf:
  data = {}
  for j, col in enumerate(row):
    data[vocabulary[j]] = col * pipe['tfid'].idf_[j]
  tfidf.append(data)

Kode ini menghitung nilai TF-IDF untuk setiap kata dalam dokumen dan menyimpannya dalam bentuk dictionary, di mana setiap dictionary mewakili satu dokumen dengan nilai TF-IDF untuk setiap kata.

### Pembuatan DataFrame

In [16]:
cat_df = main_df.loc[:, "Kategori"]
tfidf_df = pd.DataFrame(tfidf)

tfidf_df = pd.concat([cat_df, tfidf_df], axis=1)
tfidf_df

,Kategori,aam,aanisah,abadi,abai,abang,abangan,abar,abd,abdi,...,yura,yusuf,zainal,zaman,zias,zonk,zoom,zuhur,zumiatun,zunnur
0,Kuliner,68.907627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,Kuliner,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,Kuliner,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,118.12736,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,Kuliner,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,Kuliner,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Kampus,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
96,Kampus,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,4.921973,0.0,0.0,0.0,0.0
97,Kampus,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
98,Kampus,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


Kode ini menyatukan informasi kategori dengan fitur TF-IDF dari dokumen, sehingga bisa digunakan untuk analisis atau model dengan mempertimbangkan kategori dan fitur teks bersama-sama

In [17]:
tfidf_df.to_csv('data_hasil_vsm.csv', index=False)

Kode ini menyimpan hasil analisis TF-IDF beserta kategori ke dalam file CSV untuk disimpan atau digunakan di kemudian hari.